In [2]:
%pip install psycopg2 pandas

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 174.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 189.7 MB/s eta 0:00:00
  Created wheel for psycopg2: filename=psycopg2-2.9.10-cp312-cp312-linux_x86_64.whl size=552741 sha256=c6fbe177cc1e9f876086770fef94b70c849f561e132ea19aff102847a219942e
  Stored in directory: /workspace/.pyenv_mirror/pip_cache/wheels/ac/bb/ce/afa589c50b6004d3a06fc691e71bd09c9bd5f01e5921e5329b
Successfully built psycopg2

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
# 1. Connect to Postgres using Jupyter
import psycopg2
import json
import pandas as pd

# Connect to Postgres
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="postgres"
)
cursor = conn.cursor()

# 2. Create a large table with 1 million rows
# cursor.execute("""
#     CREATE TABLE large_table (
#         id SERIAL PRIMARY KEY,
#         long_text TEXT,
#         json_data JSONB
#     )
# """)

# Generate test data
import string
import random

# for i in range(1_000_000):
#     long_text = ''.join(random.choices(string.ascii_letters + string.digits, k=500))
#     json_data = {
#         "A": ''.join(random.choices(string.ascii_letters + string.digits, k=500)),
#         "B": [random.randint(1, 100) for _ in range(50)]
#     }
#     cursor.execute("INSERT INTO large_table (long_text, json_data) VALUES (%s, %s)", (long_text, json.dumps(json_data)))

# conn.commit()

# cursor.execute("CREATE EXTENSION pg_trgm")
# 3. Create indexes
cursor.execute("CREATE INDEX idx_json ON large_table USING GIN (json_data jsonb_path_ops)")
# cursor.execute("CREATE INDEX idx_text ON large_table USING GIN (long_text gin_trgm_ops)")


cursor.execute("set enable_seqscan=false")


# 4. Explain analyze for searching long text
cursor.execute("EXPLAIN  (ANALYZE ,VERBOSE ,COSTS ,BUFFERS ,TIMING ,SUMMARY,FORMAT TEXT)  SELECT * FROM large_table WHERE long_text iLIKE '%abc%'")
print("\n".join(map(str, cursor.fetchall())))

# 5. Explain analyze for searching JSON
cursor.execute("EXPLAIN  (ANALYZE ,VERBOSE ,COSTS ,BUFFERS ,TIMING ,SUMMARY,FORMAT TEXT) SELECT * FROM large_table WHERE json_data  @? '$.A like_regex \".*abc.*\"' ")
print("\n".join(map(str, cursor.fetchall())))
cursor.execute("EXPLAIN  (ANALYZE ,VERBOSE ,COSTS ,BUFFERS ,TIMING ,SUMMARY,FORMAT TEXT) SELECT * FROM large_table WHERE json_data  @? '$.B[*] ? (@ == %s )' ", (random.randint(1, 100),))
print("\n".join(map(str, cursor.fetchall())))

conn.close()

('Seq Scan on public.large_table  (cost=10000000000.00..10000262500.00 rows=40404 width=1640) (actual time=55.165..4554.262 rows=16551 loops=1)',)
('  Output: id, long_text, json_data',)
("  Filter: (large_table.long_text ~~* '%abc%'::text)",)
('  Rows Removed by Filter: 983449',)
('  Buffers: shared hit=3404 read=246596',)
('Planning:',)
('  Buffers: shared hit=55 read=16',)
('Planning Time: 2.622 ms',)
('JIT:',)
('  Functions: 2',)
('  Options: Inlining true, Optimization true, Expressions true, Deforming true',)
('  Timing: Generation 0.103 ms (Deform 0.041 ms), Inlining 36.328 ms, Optimization 10.703 ms, Emission 7.135 ms, Total 54.270 ms',)
('Execution Time: 4573.810 ms',)
('Bitmap Heap Scan on public.large_table  (cost=17260544.98..17294061.52 rows=10000 width=1640) (actual time=625.524..4643.900 rows=1000000 loops=1)',)
('  Output: id, long_text, json_data',)
('  Recheck Cond: (large_table.json_data @? \'($."A" like_regex ".*abc.*")\'::jsonpath)',)
('  Heap Blocks: exact=51743 l